# Packages and Overall Info

In [76]:
%reload_kedro

                    INFO     Resolved project path as:                                              ]8;id=404864;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=878369;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/ipython/__init__.py#132\132]8;;\
                             /home/work/Documents/repo/economic_freedom_ml.                                        
                             To set a different path, run '%reload_kedro <project_root>'                           

[02/13/23 15:28:49] INFO     Kedro project economic_freedom_ml                                      ]8;id=127169;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=445223;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/ipython/__init__.py#101\101]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=681689;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=569420;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/ipython/__init__.py#102\102]8;;\
                             'pipelines'                                                                           

In [90]:
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [77]:
model_input = catalog.load("data_with_features")

[02/13/23 15:28:59] INFO     Loading data from 'data_with_features' (CSVDataSet)...             ]8;id=706714;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=9521;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [78]:
model_input.head()

,country,year,score,property_right,government_integrity,judicial_effectiveness,tax_burden,government_spending,fiscal_health,business_freedom,...,monetary_freedom,trade_freedom,investment_freedom,financial_freedom,africa,asia,europe,north_america,oceania,south_america
0,Argentina,2023,50.1,35.1,45.1,57.9,73.3,53.0,16.8,55.1,...,37.9,60.6,55.0,60.0,0,0,0,0,0,1
1,Barbados,2023,71.3,72.6,68.7,88.2,80.6,70.8,79.7,64.7,...,78.6,58.4,70.0,60.0,0,0,0,1,0,0
2,Belize,2023,56.6,34.7,38.1,70.2,77.0,64.1,37.6,54.0,...,82.5,55.6,55.0,50.0,0,0,0,1,0,0
3,Bolivia,2023,43.0,14.1,28.7,28.4,86.0,58.8,10.9,54.7,...,72.8,60.8,15.0,40.0,0,0,0,0,0,1
4,Brazil,2023,53.3,50.3,40.0,57.2,69.9,53.8,0.4,63.2,...,78.4,60.0,60.0,50.0,0,0,0,0,0,1


In [80]:
model_input_info = pd.DataFrame({'column': model_input.columns,  
                             'dtype': model_input.dtypes,
                             'missing_percent': model_input.isna().sum()/model_input.shape[0],
                             'count': model_input.count(),
                             'n_unique': model_input.nunique()}).reset_index().drop('index', axis=1).round(decimals = 3),  
print(model_input_info)

(                    column    dtype  missing_percent  count  n_unique
0                  country   object            0.000    883        32
1                     year    int64            0.000    883        29
2                    score  float64            0.000    883       344
3           property_right  float64            0.000    883       165
4     government_integrity  float64            0.000    883       229
5   judicial_effectiveness  float64            0.746    224       147
6               tax_burden  float64            0.000    883       309
7      government_spending  float64            0.000    883       390
8            fiscal_health  float64            0.746    224       162
9         business_freedom  float64            0.000    883       325
10           labor_freedom  float64            0.325    596       353
11        monetary_freedom  float64            0.000    883       332
12           trade_freedom  float64            0.000    883       246
13      investment_

# Data Science

## Preparing the Data

### Separating the data into feature data and target data (X_all and y_all, respectively)

In [83]:
X_all = model_input.drop('score', axis=1)
y_all = model_input['score']

### Training and Testing Data Split

In [88]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.3, random_state=999)

print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 618 samples.
Testing set has 265 samples.


## Random Forest 

In [102]:
random_forest = RandomForestRegressor(random_state=999)
random_forest.fit(X_train, y_train)
rf_predicted = random_forest.predict(X_test)

In [103]:
print(r2_score(y_test,rf_predicted))

0.9999420273581918
